In [51]:
import torch
import torchvision
import torch.nn as nn
from torchvision import transforms
import torchvision.transforms as tr
import numpy as np
from torch.utils.data import DataLoader, Dataset
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import linear_sum_assignment as linear_assignment

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'{device} is available')

cuda:0 is available


In [52]:
train_data=torchvision.datasets.MNIST(root='/data',train=True,download=True,transform=tr.ToTensor())
test_data=torchvision.datasets.MNIST(root='./data',train=False,download=True,transform=tr.ToTensor())

In [53]:
#0~8번 (정상 데이터)은 normal, 9번은 abnormal(이상치)에 넣는다.
normal = []
normal_labels = []
abnormal = []
abnormal_labels = []

for i in range(60000):
  if train_data[i][1] in [0,1,2,3,4,5,6,7,8]: 
    normal.append(train_data[i][0].numpy().reshape(28,28))
    normal_labels.append(train_data[i][1])
  elif train_data[i][1] == 9:
    abnormal.append(train_data[i][0].numpy().reshape(28,28))
    abnormal_labels.append(train_data[i][1])

In [54]:
for i in range(10000):
  if test_data[i][1] in [0,1,2,3,4,5,6,7,8]: 
    normal.append(test_data[i][0].numpy().reshape(28,28))
    normal_labels.append(test_data[i][1])
  elif test_data[i][1] == 9:
    abnormal.append(test_data[i][0].numpy().reshape(28,28))
    abnormal_labels.append(test_data[i][1])

In [55]:
normal = np.array(normal)
normal_labels = np.array(normal_labels)
abnormal = np.array(abnormal)
abnormal_labels = np.array(abnormal_labels)

print("normal:",normal.shape)
print("abnormal:",abnormal.shape)
print(normal.shape[0]+abnormal.shape[0])

normal: (63042, 28, 28)
abnormal: (6958, 28, 28)
70000


In [56]:
print(np.unique(normal_labels))
print(np.unique(abnormal_labels))

[0 1 2 3 4 5 6 7 8]
[9]


In [57]:
idx=np.random.choice(63042+1,6958,replace=False) # 0이상 A 미만
trn_idx = np.delete(np.arange(63042),idx) # A 미만에서 제거 

trn_data=normal[trn_idx]
trn_label = normal_labels[trn_idx]
tes_data=np.concatenate([abnormal,normal[idx]],0)
tes_label = np.concatenate([abnormal_labels, normal_labels[idx]])
clu_tes_data = normal[idx]
clu_tes_label = normal_labels[idx]

In [58]:
#최종 training data, test data
trn_data=trn_data.reshape(-1,1,28,28)
tes_data=tes_data.reshape(-1,1,28,28)
trn_label = trn_label.reshape(-1,1)
tes_label = tes_label.reshape(-1,1)
clu_tes_data = clu_tes_data.reshape(-1,1,28,28)
clu_tes_label = clu_tes_label.reshape(-1,1)
print(trn_data.shape)
print(tes_data.shape)
print(clu_tes_data.shape) # test data 중 정상 label 데이터 6958개
print('test label:',tes_label.shape) 

(56084, 1, 28, 28)
(13916, 1, 28, 28)
(6958, 1, 28, 28)
test label: (13916, 1)


In [59]:
np.unique(tes_label,return_counts = True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 755,  874,  739,  835,  747,  717,  798,  793,  700, 6958]))

In [60]:
class TensorData(Dataset):
  def __init__(self,x_data,y_data):
    self.x_data = torch.FloatTensor(x_data)
    self.y_data = torch.LongTensor(y_data)
    self.len = len(self.x_data)

  def __getitem__(self,index):
    return self.x_data[index],self.y_data[index]

  def __len__(self):
    return self.len

In [61]:
train_data=TensorData(trn_data, trn_label)
test_data=TensorData(tes_data,tes_label)
clu_tes_data = TensorData(clu_tes_data, clu_tes_label)

train_loader = DataLoader(train_data,batch_size=128,shuffle=True)
test_loader = DataLoader(test_data,batch_size=128,shuffle=True)
clu_tes_loader = DataLoader(clu_tes_data, batch_size=128, shuffle = True)

In [62]:
#dataloader확인.
image, label = iter(train_loader).next()
print(image.shape)
image2,label2=iter(test_loader).next()
print(image2.shape,label2.shape)

torch.Size([128, 1, 28, 28])
torch.Size([128, 1, 28, 28]) torch.Size([128, 1])


### Deep k-means

In [63]:
#batch_size=128
num_clusters = 9
latent_size=10

In [64]:
class Flatten(torch.nn.Module):
  def forward(self,x):
    batch_size=x.shape[0]
    return x.view(batch_size,-1)

class Deflatten(nn.Module):
  def __init__(self,k):
    super(Deflatten,self).__init__()
    self.k=k
  def forward(self,x):
    s = x.size()
    feature_size=int((s[1]//self.k)**.5)
    return x.view(s[0],self.k,feature_size,feature_size)

In [65]:
class Kmeans(nn.Module):
  def __init__(self,num_clusters, latent_size):
    super(Kmeans,self).__init__()
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    self.num_clusters = num_clusters
    self.centroids = nn.Parameter(torch.rand((self.num_clusters,latent_size)).to(device))

  def argminl2distance(self,a,b):
    return torch.argmin(torch.sum((a-b)**2,dim=1),dim=0)

  def forward(self,x):
    y_assign = []
    for m in range(x.size(0)):
      h = x[m].expand(self.num_clusters,-1)
      assign = self.argminl2distance(h, self.centroids)
      y_assign.append(assign.item())
    return y_assign, self.centroids[y_assign]

In [66]:
class Encoder(nn.Module):
  def __init__(self,latent_size):
    super(Encoder,self).__init__()

    k=16
    self.encoder = nn.Sequential(
                      nn.Conv2d(1,k,3,stride=2),
                      nn.ReLU(),
                      nn.Conv2d(k, 2*k, 3, stride = 2),
                      nn.ReLU(),
                      nn.Conv2d(2*k, 4*k, 3, stride=1),
                      nn.ReLU(),
                      Flatten(),
                      nn.Linear(1024, latent_size),
                      nn.ReLU())
    
  def forward(self, x):
    return self.encoder(x)

class Decoder(nn.Module):
  def __init__(self, latent_size):
    super(Decoder,self).__init__()

    k=16
    self.decoder = nn.Sequential(
                      nn.Linear(latent_size,1024),
                      nn.ReLU(),
                      Deflatten(4*k),
                      nn.ConvTranspose2d(4*k, 2*k,3,stride=1),
                      nn.ReLU(),
                      nn.ConvTranspose2d(2*k, k, 3, stride = 2),
                      nn.ReLU(),
                      nn.ConvTranspose2d(k,1,3,stride=2,output_padding=1),
                      nn.Sigmoid())
    
  def forward(self,x):
    return self.decoder(x)
    

In [67]:
def cluster_acc(y_true,y_pred):
  y_true = np.array(y_true)
  y_pred = np.array(y_pred)
  D = max(y_pred.max(), y_true.max()) + 1
  w = np.zeros((D,D), dtype = np.int64)
  for i in range(y_pred.size):
    w[y_pred[i],y_true[i]] +=1
  ind = linear_assignment(w.max() - w)
  return sum([w[i,j] for i,j in zip(ind[0],ind[1])]) * 1.0 / y_pred.size

In [68]:
def evaluation(testloader, encoder, kmeans, device):
  predictions = []
  actual = []

  with torch.no_grad():
    for images, labels in testloader:
      inputs = images.to(device)
      labels = labels.to(device)
      latent_var = encoder(inputs)
      y_pred,_ = kmeans(latent_var)

      predictions+=y_pred
      actual+=labels.cpu().tolist()

  return cluster_acc(actual,predictions)

### T1 = 100

In [69]:
encoder = Encoder(latent_size).to(device)
decoder = Decoder(latent_size).to(device)
kmeans = Kmeans(num_clusters, latent_size).to(device)
criterion1 = torch.nn.MSELoss()
criterion2 = torch.nn.MSELoss()
optimizer = torch.optim.Adam(list(encoder.parameters())+list(decoder.parameters())+list(kmeans.parameters()), lr=1e-3)


In [ ]:
T1=150
T2=200
lam = 1e-3
ls = 0.05

models 폴더 생성하기.

In [ ]:
for ep in range(300):
  if ep <= T1:
    alpha = lam/(T2-T1)
  else:
    alpha = lam
  
  
  running_loss = 0.0
  ####
  for images, _ in train_loader:
    inputs = images.to(device)
    optimizer.zero_grad()
    latent_var = encoder(inputs)
    _, centroids = kmeans(latent_var.detach())
    outputs = decoder(latent_var)

    l_rec = criterion1(inputs,outputs)
    l_clt = criterion2(latent_var,centroids)
    loss = l_rec + alpha*l_clt

    loss.backward()
    optimizer.step()
    running_loss+=loss.item()
  
  ###
  avg_loss = running_loss/len(train_loader)

  if(ep%10==0):
    ###
    testacc=evaluation(clu_tes_loader,encoder,kmeans,device)
    print('[%d] Train loss: %.4f, Test Accuracy:%.3f' %(ep,avg_loss,testacc))

  # 기준 loss(hyper parameter) 보다 작게되면 신경망, kmeans의 매개변수들을 save
  if avg_loss < ls:
    ls=avg_loss

    torch.save(encoder.state_dict(),'./models/dkm_en.pth')
    torch.save(decoder.state_dict(),'./models/dkm_de.pth')
    torch.save(kmeans.state_dict(),'./models/dkm_clt.pth')

[0] Train loss: 0.0715, Test Accuracy:0.202
[10] Train loss: 0.0230, Test Accuracy:0.159
[20] Train loss: 0.0209, Test Accuracy:0.286
[30] Train loss: 0.0201, Test Accuracy:0.445
[40] Train loss: 0.0197, Test Accuracy:0.543
[50] Train loss: 0.0193, Test Accuracy:0.608
[60] Train loss: 0.0191, Test Accuracy:0.648
[70] Train loss: 0.0189, Test Accuracy:0.733
[80] Train loss: 0.0188, Test Accuracy:0.831
[90] Train loss: 0.0186, Test Accuracy:0.832
[100] Train loss: 0.0185, Test Accuracy:0.835
[110] Train loss: 0.0184, Test Accuracy:0.878
[120] Train loss: 0.0183, Test Accuracy:0.880
[130] Train loss: 0.0183, Test Accuracy:0.883
[140] Train loss: 0.0182, Test Accuracy:0.882
[150] Train loss: 0.0181, Test Accuracy:0.882
[160] Train loss: 0.0182, Test Accuracy:0.888
[170] Train loss: 0.0181, Test Accuracy:0.891
[180] Train loss: 0.0181, Test Accuracy:0.891
[190] Train loss: 0.0180, Test Accuracy:0.892
[200] Train loss: 0.0180, Test Accuracy:0.891
[210] Train loss: 0.0179, Test Accuracy:0.891

In [ ]:
import pandas as pd
idx = pd.DataFrame(idx)
idx.to_csv('test_index.csv')

## 신경망 + clustering 학습 완료

In [70]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [73]:
#forward train data.
#훈련 데이터들을 이용해 OneclassSVM을 학습시키기 위함.

encoder.load_state_dict(torch.load('/content/drive/MyDrive/캡스톤/models(mnist)/model3/dkm_en.pth'))
decoder.load_state_dict(torch.load('/content/drive/MyDrive/캡스톤/models(mnist)/model3/dkm_de.pth'))
kmeans.load_state_dict(torch.load('/content/drive/MyDrive/캡스톤/models(mnist)/model3/dkm_clt.pth'))

predictions_trn = [] # 가장 가까운 index들 모아둔 것
actual_trn = [] # 실제 label 값 모아둔 것
latent_features_trn = [] # latent vector 모아둔 것 ex) [0.0, 0.9820356369018555, 0.0, 0.4858146011829376, 0.46588805317878723, 2.642456293106079, 2.1144795417785645, 0.0, 1.0553699731826782, 0.0]


with torch.no_grad():
  for images, labels in train_loader:
    inputs = images.to(device)
    labels = labels.to(device)
    latent_var = encoder(inputs)
    y_pred, _= kmeans(latent_var)

    predictions_trn+=y_pred
    latent_features_trn+=latent_var.cpu().tolist()
    actual_trn+=labels.cpu().tolist()

print(cluster_acc(actual_trn,predictions_trn))

0.8940339490763854


In [75]:
# forward test data
# test data에 대한 
encoder.load_state_dict(torch.load('/content/drive/MyDrive/캡스톤/models(mnist)/model3/dkm_en.pth'))
decoder.load_state_dict(torch.load('/content/drive/MyDrive/캡스톤/models(mnist)/model3/dkm_de.pth'))
kmeans.load_state_dict(torch.load('/content/drive/MyDrive/캡스톤/models(mnist)/model3/dkm_clt.pth'))

predictions_tes = []
actual_tes = []


latent_features_tes = [] # svm 

with torch.no_grad():
  for images, labels in test_loader:
    inputs = images.to(device)
    labels = labels.to(device)
    latent_var = encoder(inputs)
    y_pred,_= kmeans(latent_var)

    predictions_tes+=y_pred
    latent_features_tes+=latent_var.cpu().tolist()
    actual_tes+=labels.cpu().tolist()

print(latent_features_tes[0])
print(cluster_acc(actual_tes,predictions_tes))

[0.0, 0.8471150398254395, 0.0, 1.1381382942199707, 0.9367014765739441, 1.3797794580459595, 0.23078462481498718, 0.0, 2.026674509048462, 0.0]
0.7315320494394941


In [ ]:
predictions_trn = np.array(predictions_trn)
actual_trn = np.array(actual_trn)
latent_features_trn = np.array(latent_features_trn)

predictions_tes = np.array(predictions_tes)
actual_tes = np.array(actual_tes)
latent_features_tes = np.array(latent_features_tes)

In [ ]:
import pandas as pd

latent_feature_trn=pd.DataFrame(latent_features_trn)
actual_trn = pd.DataFrame(actual_trn)
predictions_trn = pd.DataFrame(predictions_trn)

latent_feature_tes=pd.DataFrame(latent_features_tes)
actual_tes = pd.DataFrame(actual_tes)
predictions_tes = pd.DataFrame(predictions_tes)

In [ ]:
# csv 파일로 옮기는 과정
# latent_feature_trn.to_csv('/content/drive/MyDrive/캡스톤/latent/latent_features_trn.csv')
# actual_trn.to_csv('/content/drive/MyDrive/캡스톤/latent/actual_trn.csv')
# predictions_trn.to_csv('/content/drive/MyDrive/캡스톤/latent/predictions_trn.csv')

# latent_feature_tes.to_csv('/content/drive/MyDrive/캡스톤/latent/latent_features_tes.csv')
# actual_tes.to_csv('/content/drive/MyDrive/캡스톤/latent/actual_tes.csv')
# predictions_tes.to_csv('/content/drive/MyDrive/캡스톤/latent/predictions_tes.csv')

## One class svm ensemble

In [ ]:
latent_feature_trn = latent_feature_trn.values
predictions_trn = predictions_trn.values.reshape(-1,)
latent_feature_tes = latent_feature_tes.values
predictions_tes = predictions_tes.values.reshape(-1,)
actual_tes = actual_tes.values.reshape(-1,)

In [ ]:
y_true = np.where(actual_tes == 9,1,0) # 9일때는 1, 아닌 것은 0
np.unique(y_true, return_counts = True)


(array([0, 1]), array([6958, 6958]))

In [ ]:
np.unique(predictions_trn,return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8]),
 array([6010, 3931, 7677, 6168, 6061, 5686, 7386, 6348, 6817]))

In [ ]:
#OneclassSVM Ensemble
from sklearn.svm import OneClassSVM
import numpy as np
import pandas as pd

class OCSVMEnsemble():

  def __init__(self,nu=0.5):
    self.nu = nu
    #self.gamma = gamma

  def fit(self, latent, pred):
    #oneclasssvm 인스턴스가 들어갈 리스트.
    self.instance = []
    self.num_cluster = len(np.unique(pred))

    # 경계가 9개 생긴거? 
    for clu in range(self.num_cluster):
      idx = np.where(pred == clu) # 0-9 까지의 각 그룹에 해당하는 index 도출 
      #해당 군집에 속한 data
      clu_data = latent[idx]
      ocsvm = OneClassSVM(kernel = 'rbf',gamma = 'scale', nu = self.nu).fit(clu_data)
      self.instance.append(ocsvm)

  def predict(self,x):
    predict = 1
    for md in self.instance:
      #print(md)
      md_pred = md.predict(x)
      #print(md_pred)
      if md_pred == 1:
        return 0
    return predict


In [ ]:
# 학습 과정
ocsvm = OCSVMEnsemble(nu=0.5)
ocsvm.fit(latent_feature_trn,predictions_trn) # latent vector list 와 가장 가까운 index list

In [ ]:
# test과정에서의 latent vector 들을 넣어가면서 예측하는 과정
y_pred = []
for i in latent_feature_tes:
  #print(i)
  i=i.reshape(1,-1)
  pred=ocsvm.predict(i)
  y_pred.append(pred)

In [ ]:
np.unique(y_pred, return_counts = True)

(array([0, 1]), array([6203, 7713]))

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
print(classification_report(y_true,y_pred))

              precision    recall  f1-score   support

           0       0.56      0.51      0.53      6958
           1       0.55      0.61      0.58      6958

    accuracy                           0.56     13916
   macro avg       0.56      0.56      0.55     13916
weighted avg       0.56      0.56      0.55     13916

